In [1]:
import multiprocess as mp
import tqdm
import geopandas as gpd
import pandas as pd
import shapely
import fiona
shapely.speedups.enable()

### To do
- reduce coordinate precision.
- use vectorisation to loop over centroids

Load in the isochrones for the East Midlands

In [3]:
df = gpd.read_file('isochrones/west_midlands.geojson')

In [4]:
df

,iso_id,id,min,max,center,geometry
0,1,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.75444 55.53544, -3.05219 55..."
1,2,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.73947 55.53005, -3.02772 55..."
2,3,1,0.0,240.0,120.0,"MULTIPOLYGON (((-3.12151 55.52005, -3.44367 55..."
3,4,1,0.0,240.0,120.0,"MULTIPOLYGON (((-3.18137 55.52108, -3.50353 55..."
4,5,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.71318 55.47411, -1.93273 55..."
...,...,...,...,...,...,...
9622,9623,1,0.0,240.0,120.0,"MULTIPOLYGON (((-0.90456 54.23704, -1.22671 54..."
9623,9624,1,0.0,240.0,120.0,"MULTIPOLYGON (((-0.89004 54.17918, -1.21220 54..."
9624,9625,1,0.0,240.0,120.0,"MULTIPOLYGON (((-0.87553 54.21896, -1.19768 54..."
9625,9626,1,0.0,240.0,120.0,"MULTIPOLYGON (((-3.08654 54.26495, -3.10221 54..."


Load in the centroids from the population grid

In [5]:
centroids = gpd.read_file('data/gb_centroid.geojson')

In [6]:
centroids

,grid_id,population,geometry
0,1,24,POINT (-4.44691 53.42219)
1,2,15,POINT (-4.43188 53.42250)
2,3,10,POINT (-4.38678 53.42341)
3,4,32,POINT (-4.49149 53.41228)
4,5,121,POINT (-4.47646 53.41259)
...,...,...,...
117804,117805,7,POINT (-4.90856 54.66161)
117805,117806,0,POINT (-4.89308 54.66198)
117806,117807,2,POINT (-4.89244 54.65300)
117807,117808,8,POINT (-4.89180 54.64403)


Calculate the total population of the UK

In [52]:
totalpop=centroids['population'].sum()
totalpop

60553399

In [5]:
# define a function that takes a isochrone and loops through all the centroids to see if that centroid is within the isochrone. If it does, it adds the population to a running total.
def inthegoldentriangle(isochrone):
#     print(isochrone)
    population=0
    for i,centroid in centroids.iterrows():
        if(centroid['geometry'].within(isochrone)):
            population=population+centroid['population']
    if(population/totalpop>0.9):
        return True
    else:
        return False
    

In [50]:
# define a function that takes a isochrone, filters the centroids within the isochrone and then adds the population.
def inthegoldentriangle2(isochrone):
    within=centroids.loc[centroids.within(isochrone)]
    if(within['population'].sum()/totalpop>0.9):
        return True
    else:
        return False    

In [53]:
# test it
inthegoldentriangle2(df['geometry'][4])

False

In [58]:
# run the function across all the isochrones in parallel, save it to a list called output
pool=mp.Pool(8)
output = list(tqdm.tqdm_notebook(pool.imap(inthegoldentriangle2, df['geometry']), total=len(df['geometry'])))
pool.close()
pool.join()

In [59]:
output

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [60]:
# find out how many squares are within the golden triangle
sum(x==True for x in output)

703

In [61]:
# Add the column to the isochrones
df['inside']=output

In [62]:
# Here's all the isochrones that are within the golden triangle
df[df['inside']==True]

,iso_id,id,min,max,center,geometry,inside
2555,2556,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.66973 55.35001, -2.99188 55...",True
2637,2638,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.61048 55.40953, -2.93264 55...",True
2638,2639,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.62922 55.23503, -1.89677 55...",True
2639,2640,1,0.0,240.0,120.0,"MULTIPOLYGON (((-2.58088 55.38399, -2.90303 55...",True
2640,2641,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.57001 55.24352, -1.85239 55...",True
...,...,...,...,...,...,...,...
6442,6443,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.24676 55.06536, -1.56891 55...",True
6526,6527,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.37883 55.04827, -1.70099 55...",True
6532,6533,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.29082 55.05676, -1.61297 55...",True
6627,6628,1,0.0,240.0,120.0,"MULTIPOLYGON (((-1.29087 55.04787, -1.61302 55...",True


In [66]:
# use Henry's lookup
lookup=gpd.read_file('lookups/west_midlands_lookup.geojson')
lookup['inside']=output
lookup

,iso_id,id,min,max,center,population,lon,lat,geometry,inside
0,1,1,0.0,240.0,120.0,77,-1.949058,53.204560,"MULTIPOLYGON (((-2.75444 55.53544, -3.05219 55...",False
1,2,1,0.0,240.0,120.0,72,-1.934086,53.204553,"MULTIPOLYGON (((-2.73947 55.53005, -3.02772 55...",False
2,3,1,0.0,240.0,120.0,242,-1.993973,53.195582,"MULTIPOLYGON (((-3.12151 55.52005, -3.44367 55...",False
3,4,1,0.0,240.0,120.0,43,-2.053834,53.186581,"MULTIPOLYGON (((-3.18137 55.52108, -3.50353 55...",False
4,5,1,0.0,240.0,120.0,117,-1.874254,53.186527,"MULTIPOLYGON (((-1.71318 55.47411, -1.93273 55...",False
...,...,...,...,...,...,...,...,...,...,...
9622,9623,1,0.0,240.0,120.0,69,-2.676405,51.845152,"MULTIPOLYGON (((-0.90456 54.23704, -1.22671 54...",False
9623,9624,1,0.0,240.0,120.0,5,-2.661890,51.845234,"MULTIPOLYGON (((-0.89004 54.17918, -1.21220 54...",False
9624,9625,1,0.0,240.0,120.0,27,-2.647375,51.845315,"MULTIPOLYGON (((-0.87553 54.21896, -1.19768 54...",False
9625,9626,1,0.0,240.0,120.0,0,-2.632860,51.845394,"MULTIPOLYGON (((-3.08654 54.26495, -3.10221 54...",False


In [76]:
# find the centre points of the isochrones
goldentriangle=lookup[lookup['inside']==True]
goldentriangle.to_file('westmidlands-inside.geojson',driver='GeoJSON')

In [68]:
# make a geodataframe with the points
gdf = gpd.GeoDataFrame(goldentriangle, geometry=gpd.points_from_xy(goldentriangle.lon,goldentriangle.lat))
gdf

,iso_id,id,min,max,center,population,lon,lat,geometry,inside
2555,2556,1,0.0,240.0,120.0,28,-2.186494,52.719006,POINT (-2.18649 52.71901),True
2637,2638,1,0.0,240.0,120.0,9,-2.127250,52.710095,POINT (-2.12725 52.71010),True
2638,2639,1,0.0,240.0,120.0,217,-2.112448,52.710110,POINT (-2.11245 52.71011),True
2639,2640,1,0.0,240.0,120.0,154,-2.097647,52.710123,POINT (-2.09765 52.71012),True
2640,2641,1,0.0,240.0,120.0,85,-2.053243,52.710152,POINT (-2.05324 52.71015),True
...,...,...,...,...,...,...,...,...,...,...
6442,6443,1,0.0,240.0,120.0,20,-1.729990,52.323290,POINT (-1.72999 52.32329),True
6526,6527,1,0.0,240.0,120.0,68,-1.862066,52.314527,POINT (-1.86207 52.31453),True
6532,6533,1,0.0,240.0,120.0,21,-1.774051,52.314392,POINT (-1.77405 52.31439),True
6627,6628,1,0.0,240.0,120.0,132,-1.774097,52.305402,POINT (-1.77410 52.30540),True


In [71]:
# make a convex hull around the points, convert it to a geodataframe, and then save it
triangle=gdf.unary_union.convex_hull
hull = gpd.GeoDataFrame(geometry=gpd.GeoSeries(triangle))
hull
hull.set_crs('wgs84')
hull.to_file('goldentriangle-westmidlands.geojson',driver='GeoJSON')

In [75]:
gdf.to_file('goldenpoints-westmidlands.geojson',driver='GeoJSON')

Match the golden centroid points with the grid polygon

In [11]:
goldenpoints=gpd.read_file('goldenpoints-westmidlands.geojson')

In [3]:
wmgrid=gpd.read_file('wm_grid.geojson').to_crs('wgs84')
wmgrid=wmgrid.drop(columns=['OBJECTID','RGN20CD','RGN20NM','BNG_E','BNG_N','LONG','LAT','Shape__Area','Shape__Length'])

In [4]:
# goldenIDs.extend([7986,8068,8069]) 
# Don't add in the EM ids
goldenpoints

,iso_id,id,min,max,center,population,lon,lat,inside,geometry
0,2556,1,0.0,240.0,120.0,28,-2.186494,52.719006,True,POINT (-2.18649 52.71901)
1,2638,1,0.0,240.0,120.0,9,-2.127250,52.710095,True,POINT (-2.12725 52.71010)
2,2639,1,0.0,240.0,120.0,217,-2.112448,52.710110,True,POINT (-2.11245 52.71011)
3,2640,1,0.0,240.0,120.0,154,-2.097647,52.710123,True,POINT (-2.09765 52.71012)
4,2641,1,0.0,240.0,120.0,85,-2.053243,52.710152,True,POINT (-2.05324 52.71015)
...,...,...,...,...,...,...,...,...,...,...
698,6443,1,0.0,240.0,120.0,20,-1.729990,52.323290,True,POINT (-1.72999 52.32329)
699,6527,1,0.0,240.0,120.0,68,-1.862066,52.314527,True,POINT (-1.86207 52.31453)
700,6533,1,0.0,240.0,120.0,21,-1.774051,52.314392,True,POINT (-1.77405 52.31439)
701,6628,1,0.0,240.0,120.0,132,-1.774097,52.305402,True,POINT (-1.77410 52.30540)


In [15]:
goldengrid=gpd.sjoin(wmgrid,goldenpoints, op='contains')

In [16]:
goldengrid.to_file('wmgoldengrid.geojson',driver='GeoJSON')

In [17]:
onegrid=goldengrid.dissolve(by='inside')

In [18]:
onegrid.to_file('wmgoldengridmerged.geojson',driver='GeoJSON')